# Random walk on a graph

Let $G$ be an undirected finite graph.
Let $X$ be a random walk on the vertices of $G$,
so that at each time step, if $X(t) = v$,
then $X(t+1)$ is a uniformly chosen neighbor of $v$.

1. Show that this random walk is reversible,
    and find a stationary distribution.

2. Let $P$ be the transition matrix of this random walk,
    and show that the null space of $(P - I)$ is spanned by vectors
    that are constant on each connected component of the graph.
    (*Hint:* suppose that $xP = x$ but $x$ has a local maximum.)

3. Modify the random walk so that the *uniform* distribution on vertices
    is a stationary distribution.

# Adjacency matrices and random walks

Let $A$ be the adjacency matrix of an undirected finite graph $G = (V, E)$,
i.e., $A_{ij} = 1$ if $(i, j)$ is an edge in the graph, and $A_{ij} = 0$ otherwise.

1. Let $D$ be the diagonal matrix with $D_{ii} = \text{deg}(i)$.
    Show that $P_t = \exp(t(A - D))$ is a valid transition matrix for any $t > 0$.

2. Prove that $P_t$ has the *same* stationary distributions for any $t$,
    and that any such stationary distribution $\pi$ must solve $\pi A = \pi D$.

# Traveling sales

Here is a matrix of distances between 10 cities:
```
array([[  0.,  23.,  22., 102., 100.,  18.,  79.,  26.,  47.,  84.],
       [ 23.,   0.,  45.,  81.,  79.,   7.,  75.,  26.,  29.,  62.],
       [ 22.,  45.,   0., 125., 122.,  41.,  93.,  39.,  69., 106.],
       [102.,  81., 125.,   0.,  11.,  84.,  82., 102.,  57.,  18.],
       [100.,  79., 122.,  11.,   0.,  82.,  71., 102.,  53.,  19.],
       [ 18.,   7.,  41.,  84.,  82.,   0.,  70.,  29.,  29.,  65.],
       [ 79.,  75.,  93.,  82.,  71.,  70.,   0.,  99.,  53.,  71.],
       [ 26.,  26.,  39., 102., 102.,  29.,  99.,   0.,  55.,  84.],
       [ 47.,  29.,  69.,  57.,  53.,  29.,  53.,  55.,   0.,  39.],
       [ 84.,  62., 106.,  18.,  19.,  65.,  71.,  84.,  39.,   0.]])
```
Write a simulated annealing algorithm to find the shortest tour of all cities (that returns to where it started).
Verify that the answer is correct by exhaustive search.

# The Monte Carlo's storeroom

Write an MCMC to sample from the Gibbs distribution on box stackings
with $H(S) = |S|/T$, where $|S|$ is the number of boxes in the stacking $S$,
and $T$ is the temperature.
Use the following code to animate your MCMC as a visual sanity check at both low temperature and high temperature.
Verify that the code samples from the correct distribution
using a very small example (e.g., 2x2x2).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import collections as cs
from matplotlib import animation as ani
plt.rcParams["animation.html"] = "jshtml"

In [ ]:
plt.rcParams['figure.figsize'] = [8, 8]

In [ ]:
def proj(a, b, c):
    xy = np.array(a * np.array([1/2, -1/4])
                  + b * np.array([-1/2, -1/4])
                  + c * np.array([0, 1/2]))
    return xy

def box(a, b, c, top, left, right):
    out = []
    if top:
        out.extend([(0, 0), (-1/2, 1/4), (0, 1/2), (1/2, 1/4), (0, 0)])
    if left:
        out.extend([(0, 0), (-1/2, 1/4), (-1/2, -1/4), (0, -1/2), (0, 0)])
    if right:
        out.extend([(0, 0), (1/2, 1/4), (1/2, -1/4), (0, -1/2), (0, 0)])
    out = np.array(out)
    if len(out) > 0:
        out += proj(a, b, c)
    return out

class BoxStack(object):
    
    def __init__(self, dims):
        # initialize self with no boxes
        self.dims = dims
        # this 3d array will have a 1 at [i,j,k] if there is a box there
        self.boxes = np.repeat(False, np.prod(dims)).reshape(dims)
    
    def size(self):
        return np.sum(self.boxes)
    
    def setbox(self, a, b, c, z):
        self.boxes[a, b, c] = z
    
    def box(self, a, b, c):
        return box(a, b, c,
                   top=((a >= 0 and b >= 0) and
                        ((c == self.dims[2] - 1)
                         or not self.boxes[a, b, c + 1])),
                   left=((a >= 0 and c >= 0) and
                         ((b == self.dims[1] - 1)
                          or not self.boxes[a, b + 1, c])),
                   right=((b >= 0 and c >= 0) and
                          ((a == self.dims[0] - 1)
                           or not self.boxes[a + 1, b, c]))
                  )
    
    def lines(self):
        boxen = [self.box(a, b, c) for a in range(self.dims[0])
                                   for b in range(self.dims[1])
                                   for c in range(self.dims[2])
                                   if self.boxes[a, b, c]]
        boxen += [self.box(-1, b, c) for b in range(self.dims[1])
                                     for c in range(self.dims[2])]
        boxen += [self.box(a, -1, c) for a in range(self.dims[0])
                                     for c in range(self.dims[2])]
        boxen += [self.box(a, b, -1) for a in range(self.dims[0])
                                     for b in range(self.dims[1])]
        return [u for u in boxen if len(u) > 0]
       
    def draw(self):
        fig, ax = plt.subplots()
        boxen = cs.LineCollection(self.lines())
        ax.add_collection(boxen)
        ax.set(xlim=(-1-self.dims[1]/2, 1+self.dims[0]/2), 
               ylim=(-1-max(self.dims[0], 1+self.dims[1])/2, self.dims[2]/2),
               aspect=1)


def animate_boxes(a, b, c, box_list):
    S = BoxStack((a, b, c))
    fig, ax = plt.subplots()
    boxen = cs.LineCollection(S.lines())
    ax.add_collection(boxen)
    ax.set(xlim=(-(1+S.dims[1])/2, (1+S.dims[0])/2), 
           ylim=(-(1+max(S.dims[0], S.dims[1]))/2, (1+S.dims[2])/2),
           aspect=1)
        
    def update(frame):
        (a, b, c), z = box_list[int(frame)]
        S.setbox(a, b, c, z)
        boxen.set_paths(S.lines())
        return boxen

    animation = ani.FuncAnimation(fig, update,
                                  frames=np.arange(len(box_list)))
    plt.close(animation._fig)
    return animation

In [ ]:
# example:
S = BoxStack((6, 8, 4))
S.setbox(0, 0, 0, True)
S.setbox(0, 0, 1, True)
S.setbox(0, 1, 0, True)
S.setbox(1, 0, 0, True)
S.setbox(1, 1, 0, True)
S.setbox(0, 2, 0, True)
S.draw()

In [ ]:
# another example:
boxlist = [[(0, 0, 0), True],
           [(0, 0, 1), True],
           [(0, 1, 0), True],
           [(1, 0, 0), True],
           [(1, 1, 0), True],
           [(1, 0, 1), True],
           [(0, 1, 1), True],
           [(1, 1, 1), True]]
anim = animate_boxes(4, 5, 6, boxlist + [(a, False) for a, z in reversed(boxlist)])
anim

In [ ]:
# yet another example:
boxlist = [[(a, b, c), True] for a in range(4) for b in range(5) for c in range(3)]
boxlist += list(reversed([[(a, b, c), False] for c in range(3) for b in range(5) for a in range(4)]))

anim = animate_boxes(4, 5, 6, boxlist)
anim